In [ ]:
!pip install transformers torch fitz

Carregamento do Documento

In [2]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Exemplo de uso:
pdf_path = "./Guia.pdf"
document_text = extract_text_from_pdf(pdf_path)


ModuleNotFoundError: No module named 'fitz'

Treinar o Modelo

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

# Nome do modelo pré-treinado
model_name = "neuralmind/bert-base-portuguese-cased"

# Carregar o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Função de tokenização
def tokenize_function(examples):
    return tokenizer(examples['title'], padding="max_length", truncation=True, max_length=512)

# Carregar o dataset CSV usando pandas
df = pd.read_csv('./dataset/fakes.csv')

# Converter o DataFrame para um Dataset do Hugging Face
dataset = Dataset.from_pandas(df)

# Dividir o dataset em treino e teste
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
test_dataset = dataset['test']

# Tokenizar os datasets de treino e teste
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

print(training_args)

# Definir a função de avaliação
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)

# Treinar o modelo
trainer.train()

# Avaliar o modelo
trainer.evaluate()

# Exemplo de texto para classificação
test_text = "Exemplo de notícia para verificação."

# Tokenizar o texto
inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Fazer a previsão
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

# Verificar a previsão
label = predictions.item()
label_name = "Fake" if label == 1 else "Real"
print(f"A notícia é classificada como: {label_name}")


ModuleNotFoundError: No module named 'datasets'